In [ ]:
'''
talib
'''

In [ ]:
import talib
import numpy as np

In [ ]:
high = np.array([1.0, 2.0, 6.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
low = np.array([1.0, 2.0, 5.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0])
close = np.array([1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
a = talib.ATR(high, low, close, 2)
a

In [ ]:
close = np.array([1000.0, 2.0, 1.0, 6.0, 6.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0])
result = talib.RSI(close, 2)
result

In [ ]:
'''
tushare
'''

In [15]:
import tushare as ts
from datetime import datetime, timedelta

In [35]:
df = ts.get_tick_data('rb1805', '2017-09-10')
#df

In [36]:
cons = ts.get_apis()

In [39]:
df = ts.tick('rb1805', conn = cons, date = '2017-08-09', asset = 'X') 
#df = ts.tick('IF1801', conn = cons, date = '2017-06-22', asset = 'X') 
#df = ts.tick('600380', conn = cons, date = '2018-01-02')
#df = ts.tick('rb1806', conn = cons, date = '2018-01-10', asset = 'X')
#df

In [40]:
lastDt = None
mSecond = 0
for i, row in df.iterrows():
    dt = row['date'].to_datetime()
    if (not lastDt) or lastDt != dt:
        mSecond = 0
        lastDt = dt
    else:
        mSecond += 1
    
    #print mSecond
    dt = dt.replace(microsecond = mSecond)
        
    print dt
    print row['vol']
    print '\n'

2017-08-09 20:59:00
3723.0


2017-08-09 21:00:00
3723.0


2017-08-09 21:00:01
3723.0


2017-08-09 21:00:01.000001
3723.0


2017-08-09 21:00:02
3723.0


2017-08-09 21:00:02.000001
3723.0


2017-08-09 21:00:03
3723.0


2017-08-09 21:00:03.000001
3726.0


2017-08-09 21:00:04
3727.0


2017-08-09 21:00:04.000001
3727.0


2017-08-09 21:00:05
3729.0


2017-08-09 21:00:05.000001
3730.0


2017-08-09 21:00:06
3734.0


2017-08-09 21:00:06.000001
3739.0


2017-08-09 21:00:07
3738.0


2017-08-09 21:00:07.000001
3739.0


2017-08-09 21:00:08
3738.0


2017-08-09 21:00:08.000001
3737.0


2017-08-09 21:00:09
3742.0


2017-08-09 21:00:09.000001
3742.0


2017-08-09 21:00:10
3742.0


2017-08-09 21:00:10.000001
3740.0


2017-08-09 21:00:11
3738.0


2017-08-09 21:00:11.000001
3737.0


2017-08-09 21:00:12
3737.0


2017-08-09 21:00:13
3736.0


2017-08-09 21:00:14
3736.0


2017-08-09 21:00:14.000001
3738.0


2017-08-09 21:00:17
3738.0


2017-08-09 21:00:17.000001
3734.0


2017-08-09 21:00:18
3732.0


2017-08-09

In [ ]:
for i, row in df.iterrows():
    print row['date'], '*'*2, row['vol']

In [ ]:
df = ts.bar('rb1805', conn = cons, freq = 'D', start_date = '2017-01-01', end_date = '', asset = 'X')

In [ ]:
df.sort_index(inplace = True)
df

In [ ]:
a = df.ix[1]
the = a['date']
dt = the.to_datetime()
print dt
if dt.time() >= datetime.time(21, 0):
    dt = dt - timedelta(1)

print dt

In [ ]:
for index, row in df.iterrows():
    print row['date'], ' '*2, row['vol']

In [ ]:
from datetime import datetime

In [ ]:
from collections import namedtuple
abc = namedtuple('xyz', ['x', 'y'])
the = abc(6,2)
the.x

In [ ]:
import sys
#sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding('UTF-8')
a = sys.getdefaultencoding()
type(a)

In [ ]:
if isinstance('a', str):
    print 1
else:
    print 2

In [ ]:
import pymongo 

In [ ]:
c = pymongo.MongoClient('localhot', 270)
if c:
    print c
    
else:
    print 'no'

In [ ]:
'''
datetime
'''

In [ ]:
from datetime import datetime, time

In [ ]:
a = datetime.now().time()
print type(a)

In [ ]:
a = False
if not a:
    print 'yes'

In [ ]:
'''
json
'''
import json

In [ ]:
f1 = open(filePath)
the = json.load(f1)
for dic in the:
    print dic['a']

In [ ]:
import os

In [ ]:
currentFolder = os.getcwd()
print currentFolder
newPath = os.path.join(currentFolder, '')

In [31]:
d = datetime.now().strftime('Y-m-d')
d

'Y-m-d'

In [ ]:
'''
pandas
'''

In [49]:
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [64]:
b = DataFrame()
b['名称'] = []
b['价格'] = []
b['开盘价'] = []
b.ix['2017-01-01'] = [1,2,3]
b.ix['2017-01-02'] = [1,2,3]
b.ix['2017-01-03'] = [1,2,3]
b

,名称,价格,开盘价
2017-01-01,1.0,2.0,3.0
2017-01-02,1.0,2.0,3.0
2017-01-03,1.0,2.0,3.0


In [65]:
fileName = 'abc.csv'

In [66]:
b.to_csv(fileName, encoding = 'utf-8')

In [67]:
c = pd.read_csv(fileName, encoding = 'utf-8')
c

,Unnamed: 0,名称,价格,开盘价
0,2017-01-01,1.0,2.0,3.0
1,2017-01-02,1.0,2.0,3.0
2,2017-01-03,1.0,2.0,3.0


In [ ]:
'''
csv
'''

In [37]:
import csv
import codecs

In [39]:
fileName = 'abc.csv'

In [54]:
# 写
f = file(fileName, 'wb')
f.write(codecs.BOM_UTF8)

fieldnames = [u'a', 'last_name']
writer = csv.DictWriter(f, fieldnames=fieldnames)

writer.writeheader()
writer.writerow({'a': '和', 'last_name': '啊'})

In [56]:
# 读
with open(fileName) as f:
    reader = csv.DictReader(f)
    the = []
    for row in reader:
        the.append(row)
    print the

[]


In [ ]:
'''
os
'''

In [5]:
import os

In [20]:
exists = os.path.exists(fileName)
print exists
print os.getcwd()
newFileName = os.getcwd() + '\clone\\abc.csv'
print newFileName
print os.path.exists(newFileName)

True
C:\Users\loe\Quant\python-for-investment\jupyter
C:\Users\loe\Quant\python-for-investment\jupyter\clone\abc.csv
True


In [28]:
dic = {'a':1, 'b':2, 'c':3}
print dic.keys()
print type(dic.keys())

['a', 'c', 'b']
<type 'list'>


In [13]:
walkingDic = 'C:\\Users'
for root, subdirs, files in os.walk(walkingDic):
    for sub in subdirs:
        if sub == 'stgEarningCSV':
            print root

C:\Users\Rachel\iCloudDrive\com~apple~Numbers\ctaStrategy
yes


In [ ]:
'''
MongoDB
'''

In [2]:
import pymongo
from datetime import datetime
from vnpy.trader.app.ctaStrategy.ctaBase import TICK_DB_NAME

In [25]:
code = 'rb1805.SHFE'
#code = 'rb1805'
startDate = datetime.strptime('2018-01-11 22:59:50', '%Y-%m-%d %H:%M:%S')
endDate = datetime.strptime('2018-01-11 22:59:59', '%Y-%m-%d %H:%M:%S')
client = pymongo.MongoClient('localhost', 27017)
collection = client[TICK_DB_NAME][code]
flt = {'datetime':{'$gte':startDate,
                   '$lt':endDate}}        
data = collection.find(flt).sort('datetime')
for dic in data:
    print dic['datetime']
    print dic['lastPrice']
    print '\n'

2018-01-11 22:59:50
3823.0


2018-01-11 22:59:50.001000
3823.0


2018-01-11 22:59:51
3824.0


2018-01-11 22:59:51.001000
3824.0


2018-01-11 22:59:52
3823.0


2018-01-11 22:59:52.001000
3823.0


2018-01-11 22:59:53
3823.0


2018-01-11 22:59:53.001000
3822.0


2018-01-11 22:59:54
3820.0


2018-01-11 22:59:54.001000
3821.0


2018-01-11 22:59:55
3817.0


2018-01-11 22:59:55.001000
3815.0


2018-01-11 22:59:56
3815.0


2018-01-11 22:59:56.001000
3811.0


2018-01-11 22:59:57
3810.0


2018-01-11 22:59:57.001000
3811.0


2018-01-11 22:59:58
3812.0


2018-01-11 22:59:58.001000
3813.0




In [24]:
code = 'rb1805'
startDate = datetime.strptime('2018-01-11 22:59:50', '%Y-%m-%d %H:%M:%S')
endDate = datetime.strptime('2018-01-11 22:59:59', '%Y-%m-%d %H:%M:%S')
client = pymongo.MongoClient('localhost', 27017)
collection = client[TICK_DB_NAME][code]
flt = {'datetime':{'$gte':startDate,
                   '$lt':endDate}}        
data = collection.find(flt).sort('datetime')
for dic in data:
    print dic['datetime']
    print dic['lastPrice']
    print '\n'

2018-01-12 11:26:39
3795.0


2018-01-12 11:26:39.500000
3794.0


2018-01-12 11:26:40
3794.0


2018-01-12 11:26:40.500000
3794.0


2018-01-12 11:26:41
3794.0


2018-01-12 11:26:41.500000
3795.0


2018-01-12 11:26:42
3794.0


2018-01-12 11:26:42.500000
3794.0


2018-01-12 11:26:43
3795.0


2018-01-12 11:26:43.500000
3795.0


2018-01-12 11:26:44
3794.0


2018-01-12 11:26:44.500000
3795.0


2018-01-12 11:26:45
3795.0


2018-01-12 11:26:45.500000
3794.0


2018-01-12 11:26:46
3794.0


2018-01-12 11:26:46.500000
3794.0


2018-01-12 11:26:47
3794.0


2018-01-12 11:26:47.500000
3794.0


2018-01-12 11:26:48
3794.0


2018-01-12 11:26:48.500000
3795.0


2018-01-12 11:26:49
3794.0


2018-01-12 11:26:49.500000
3795.0


2018-01-12 11:26:50
3794.0


2018-01-12 11:26:50.500000
3794.0


2018-01-12 11:26:51.500000
3794.0


2018-01-12 11:26:52
3794.0


2018-01-12 11:26:52.500000
3794.0


2018-01-12 11:26:53
3795.0


2018-01-12 11:26:53.500000
3794.0


2018-01-12 11:26:54
3794.0


2018-01-12 11:26:54.50000